In [9]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

# Define paramaters for the model
learning_rate = 0.5
n_epochs = 10
hidden_size = 10
# regulation_rate = 1e-4

def fullLayer(input_data, output_size, act, std = 0.1):
    
    ##defining the full linear Layer here
    w = tf.Variable(tf.random_normal([input_data.get_shape().as_list()[1], output_size], stddev = std))
#     w = tf.Variable(tf.zeros([input_data.shape[1], output_size]))
    b = tf.Variable(tf.zeros([output_size]))
    return act(tf.matmul(input_data, w) + b)

#read data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)



for batch_size in [1, 10, 50, 100, 1000]:
    
    tf.reset_default_graph()
    
    #define placeholder
    # all image are 28 * 28 so x has 784 dimension
    X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder')
    Y = tf.placeholder(tf.float32, [batch_size, 10], name='Y_placeholder')

    ## the layers
    h = fullLayer(X, hidden_size, tf.nn.sigmoid)
    logits = fullLayer(h, 10, tf.nn.sigmoid)

    ## defining loss function
    ## use cross entropy of softmax of logits as the loss function
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
    ## computes the mean over all the examples in the batch
    loss = tf.reduce_mean(entropy) 
    # + regulation_rate*tf.nn.l2_loss(w)  

    ##defining optimizer
    ## using gradient descent with learning rate of 0.5 to minimize loss
    gradient = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = gradient.minimize(loss)

    ##the prediction we made
    preds = tf.nn.softmax(logits)
    ##check how many of them are correct arg maxx is used because Y is one hat
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    
    #just some config for not getting whole server
    gpu_options = tf.GPUOptions(allow_growth=True)
    config = tf.ConfigProto(gpu_options=gpu_options)
    
    with tf.Session(config = config) as sess:
        
        # to visualize using TensorBoard
        tf.summary.scalar("loss__batch__size__" + str(batch_size), loss)
        merge=tf.summary.merge_all()
        writer=tf.summary.FileWriter("./graphs/mnist/b_all")

        ##starting time
        start_time = time.time()
        ##initialize the variables
        sess.run(tf.global_variables_initializer())
        
        ##number of train batches
        n_batches = int(mnist.train.num_examples / batch_size)
        
        for i in range(batch_size):  
            ##because no matter what batch_size vector computations take similar time 
            ##it is logical that number of epochs must be linearly dependant to batch_size
            ##so if number of epoches be batch_size all of batch_sizes proccesses can take similar time
            total_loss = 0
            total_acc = 0
            for _ in range(n_batches):
                ##train batches
                X_batch, Y_batch = mnist.train.next_batch(batch_size)
                
                _, loss_batch, acc_batch = sess.run([optimizer, loss, accuracy], feed_dict={X: X_batch, Y: Y_batch})
                total_loss += loss_batch
                total_acc += acc_batch

            if i % max(batch_size // 10, 1) == 0:
                print('with batch_size {}, epoch {}, Average loss : {}, Accuracy : {:.6f}'.format(batch_size, i, total_loss / n_batches, total_acc / mnist.train.num_examples))

        print('Total time: {0} seconds'.format(time.time() - start_time))
        print('Optimization Finished!')  # should be around 0.35 after 25 epochs

        ## test the model
        ## number of test batches
        n_batches = int(mnist.test.num_examples / batch_size)
        total_correct_preds = 0
        for i in range(n_batches):
            ##test batches
            X_batch, Y_batch = mnist.test.next_batch(batch_size)
            accuracy_batch, m = sess.run([accuracy, merge], feed_dict={X: X_batch, Y: Y_batch})
            total_correct_preds += accuracy_batch
            
            writer.add_summary(m, i)

        print('Accuracy batch_size {}, {}'.format(batch_size, total_correct_preds / mnist.test.num_examples))

        writer.close()
    sess.close()

'''
وقتی سایز بچ 1 است مقدار گرادیان محاسبه شده تقریب بسیار ضعیفی از گرادیان است از این رو 
مقدار لاس به صورت میانگین برای 1 خیلی کم تر است اما برای 10 و 50 و 100 تقریبا مشابه عمل می کند
و برای روش بچ کامل نیز چون تعداد ایپاک ها زیاد است زیاد میانگین لاس فرق نمیکند
اما توجه داشته باشید چون تعداد ایپاک ها را سایز بچ در نظر گرفتیم هر چه بچ سایز بزرگ تر بوده زمان بیشتری یادگیری طول می کشد
اما زمان یادگیری برای سایز های 1 تا 100 چون روی جی پی یو است زیاد تفاوت نمیکنند
اما برای بچ سایز1000 با اینکه تعداد بار هایی که گرادیان را محاسبه میکنیم یکسان است خیلی بیشتر طول میکشد
بیشتر از سه برابر بیشتر طول میکشد
توجه کنید که مهم بهتر عمل کردن در زمان کم تر است
نتیجه گیری کلی یکی این است که 10 و 50 و 100 بهتر عمل میکنند
'''   


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
with batch_size 1, epoch 0, Average loss : 1.6082455511461604, Accuracy : 0.820436
Total time: 127.64351558685303 seconds
Optimization Finished!
Accuracy batch_size 1, 0.8787
with batch_size 10, epoch 0, Average loss : 1.7089301073551177, Accuracy : 0.796891
with batch_size 10, epoch 1, Average loss : 1.5909484843124042, Accuracy : 0.817455
with batch_size 10, epoch 2, Average loss : 1.5739139389558272, Accuracy : 0.826382
with batch_size 10, epoch 3, Average loss : 1.5657847174947912, Accuracy : 0.843491
with batch_size 10, epoch 4, Average loss : 1.5607086818001488, Accuracy : 0.867145
with batch_size 10, epoch 5, Average loss : 1.5573890530846335, Accuracy : 0.895727
with batch_size 10, epoch 6, Average loss : 1.5545648713978855, Accuracy : 0.904927
with batch_size 10, epoch 7, Average loss :

'\nمقدار لاس به صورت میانگین برای 1 کم تر است اما برای 10 و 50 و 100 تقریبا مشابه عمل می کند\nو برای روش بچ کامل نیز چون تعدا ایپاک ها زیاد است زیاد میانگین لاس فرق نمیکند\nاما توجه داشته باشید چون تعداد ایپاک ها را سایز بچ در نظر گرفتیم هر چه بچ سایز بزرگ تر بوده زمان بیشتری یادگیری طول می کشد\nاما زمان یادگیری برای سایز های 1 تا 100 چون روی جی پی یو است زیاد تفاوت نمیکند اما برای 1000 خیلی بیشتر طول میکشد\nنتیجه گیری کلی یکی این است که 10 و 50 و 100 بهتر عمل میککند\n'